## Experiments on prompting with Groq and LlamaIndex

Groq - for deterministic and low latency execution. Need an API key (Open source)

Zero shot prompting - Passing instructions, context and queries without any examples. Used when model can answer freely but within the context. Style of answer may vary. 


In [4]:
! pip install -q -U llama-index llama-index-llms-groq


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Create an API Key From Groq

In [11]:
import os
from llama_index.core  import PromptTemplate
from llama_index.llms.groq import Groq
from dotenv import load_dotenv

load_dotenv()
api_key = os.environ["GROQ_API_KEY"]

# Zero shot prompting

In [16]:
# Instruction is created once. This function can be called for any context and query
def zero_shot_prompt(context, query, api_key):
    llm = Groq(
        api_key=api_key, model = "llama-3.3-70b-versatile", temperature=0
    )
    template_str = (
        "You are helpful assistant.\n"
        "Use only the context provided and answer the user's query."
        "If the context is not sufficient, reply 'Not enough information' \n "
        "Context:\n{context_str}\n"
        "User Query:\n{query_str}\n"
        "Rules for answering : \n"
        "1. Answer in 2-3 sentances precisely.\n"
        "2. Use bullet points for list.\n\n"
        "Answer to the Query:"
    )

    prompt = PromptTemplate(template_str).format(context_str=context, query_str=query)
    response = llm.complete(prompt)
    output_str = response.text
    return output_str

In [17]:
# sample 1
context_text = (
    "Transformers use a self-attention mechanism that allows each token "
    "to attend to all other tokens in the sequence. This helps capture "
    "long-range dependencies without recurrence."
)

query_text = "How do Transformers handle long-range dependencies?"

answer_str = zero_shot_prompt(context=context_text, query=query_text, api_key=api_key)

print(answer_str)

Transformers handle long-range dependencies using a self-attention mechanism. This mechanism allows each token to attend to all other tokens in the sequence. Key benefits of this approach include:
* Capturing dependencies without recurrence, enabling more efficient processing.


In [9]:
# sample 2
context_text = (
    "A knowledge graph stores information as entities and relationships." 
    "In a factory setting, entities include machines, components, and "
    "defects. Relationships describe how entities are connected, such as " 
    "machine uses component or component causes defect. By linking defects "
    "to specific components and machines, the knowledge graph enables tracing "
    "connections across the production process. This structured representation "
    "helps teams understand how problems propagate in the system."
)
query_text = "How a knowledge graph helps trace the cause of a defect in a factory."
answer_str = zero_shot_prompt(context=context_text, query=query_text)
print(answer_str)

A knowledge graph helps trace the cause of a defect in a factory by linking defects to specific components and machines. This is achieved through relationships such as:
* component causes defect
* machine uses component. By doing so, teams can understand how problems propagate in the system and identify the root cause of a defect.


Few shot prompting

In [6]:
# Instruction is created once. This function can be called for any context, query and example

def few_shot_prompt(context, query, example):
    llm = Groq(
        model = "llama-3.3-70b-versatile", temperature = 0
    )
    template_str = (
        "You are helpful assistant.\n"
        "Use only the context provided and answer the user's query."
        "If the context is not sufficient, reply 'Not enough information' \n "
        "Follow the examples provided to generate the resule"
        "Examples:\n{example_str}\n"
        "Context:\n{context_str}\n"
        "User Query:\n{query_str}\n"
        "Rules for answering : \n"
        "1. Answer in 2-3 sentances precisely.\n"
        "2. Use bullet points for list.\n\n"
        "3) At the end, include a 'Sources:' section with short snippets or filenames from the context you used.\n\n"
        "Answer to the Query:"
    )

    prompt = PromptTemplate(template_str).format(example_str=example, context_str= context, query_str=query)
    response = llm.complete(prompt)
    return response.text

In [8]:
# Few-Shot: Add examples so the model mimics your style
shots = [
    {
        "context": "Positional encodings inject order information into sequences.",
        "question": "Why are positional encodings needed?",
        "answer": (
            "They give the model a sense of word order.\n"
            "- Without them, the model treats tokens as a bag of words.\n"
            "- Encodings ensure the sequence structure is preserved.\n"
            "Sources: lecture_notes.txt"
        )
    },
    {
        "context": "Multi-head attention projects queries, keys, and values into multiple subspaces.",
        "question": "What is the benefit of multi-head attention?",
        "answer": (
            "It lets the model learn from different representation subspaces.\n"
            "- Captures diverse relationships.\n"
            "- Improves contextual understanding.\n"
            "Sources: attention_paper.pdf"
        )
    },
]

In [9]:
context_text = (
    "A knowledge graph stores information as entities and relationships." 
    "In a factory setting, entities include machines, components, and "
    "defects. Relationships describe how entities are connected, such as " 
    "machine uses component or component causes defect. By linking defects "
    "to specific components and machines, the knowledge graph enables tracing "
    "connections across the production process. This structured representation "
    "helps teams understand how problems propagate in the system."
)
query_text = "How a knowledge graph helps trace the cause of a defect in a factory."
answer_str = few_shot_prompt(context=context_text, query=query_text, example=shots)
print(answer_str)

A knowledge graph helps trace the cause of a defect in a factory by linking defects to specific components and machines. This is achieved through relationships such as:
* machine uses component
* component causes defect
By doing so, it enables teams to understand how problems propagate in the system. 
Sources: knowledge_graph_description.txt
